<a href="https://colab.research.google.com/github/wose70/HDB_Rental/blob/main/HDB_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import folium
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from folium.plugins import MarkerCluster

# Load the datasets
rent_data = pd.read_csv("RentingOutofFlats2025.csv")
coordinates_data = pd.read_csv("street_coordinates.csv")

# Merge the datasets on a common key (e.g., street name or equivalent column)
data = pd.merge(rent_data, coordinates_data, how="left", on="street_name")

# Data cleaning (e.g., handling missing values)
data.dropna(subset=["longitude", "latitude", "monthly_rent"], inplace=True)

# Scale the rental prices for clustering
scaler = StandardScaler()
data["scaled_rent"] = scaler.fit_transform(data[["monthly_rent"]])

# Apply K-Means clustering
n_clusters = 5  # You can adjust the number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
data["cluster"] = kmeans.fit_predict(data[["scaled_rent", "longitude", "latitude"]])

# Create a Folium map
map_center = [data["latitude"].mean(), data["longitude"].mean()]  # Center of the map
rental_map = folium.Map(location=map_center, zoom_start=12)

# Add clusters to the map
marker_cluster = MarkerCluster().add_to(rental_map)

# Add markers for each rental point
for idx, row in data.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=f"Monthly Rent: ${row['monthly_rent']}, Cluster: {row['cluster']}",
        icon=folium.Icon(color="blue" if row["cluster"] == 0 else "green")
    ).add_to(marker_cluster)

# Save the map to an HTML file
rental_map.save("rental_price_clusters_map.html")

print("Map has been saved as 'rental_price_clusters_map.html'")

Map has been saved as 'rental_price_clusters_map.html'
